In [1]:
import torchvision.models as models
import torch.onnx as onnx
import torch

/opt/conda/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import random
import time
import PIL
import cv2
import os

In [3]:
from models.experimental import attempt_load
from utils.datasets import letterbox
from utils.general import check_img_size, non_max_suppression_face, apply_classifier, scale_coords, xywh2xyxy, xyxy2xywh, \
    strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized

In [4]:
def make_dir(dirName):
    # Create a target directory & all intermediate 
    # directories if they don't exists
    
    if not os.path.exists(dirName):
        os.makedirs(dirName, exist_ok = True)
        print("[INFO] Directory " ,dirName,  " created")
    else:
        print("[INFO] Directory " ,dirName,  " already exists") 

In [5]:
def scale_coords_landmarks(img1_shape, coords, img0_shape, ratio_pad=None):
    # Rescale coords (xyxy) from img1_shape to img0_shape
    
    if ratio_pad is None:  # calculate from img0_shape
        gain = min(img1_shape[0] / img0_shape[0], img1_shape[1] / img0_shape[1])  # gain  = old / new
        pad = (img1_shape[1] - img0_shape[1] * gain) / 2, (img1_shape[0] - img0_shape[0] * gain) / 2  # wh padding
    else:
        gain = ratio_pad[0][0]
        pad = ratio_pad[1]

    coords[:, [0, 2, 4, 6, 8]] -= pad[0]  # x padding
    coords[:, [1, 3, 5, 7, 9]] -= pad[1]  # y padding
    coords[:, :10] /= gain
    coords[:, 0].clamp_(0, img0_shape[1])  # x1
    coords[:, 1].clamp_(0, img0_shape[0])  # y1
    coords[:, 2].clamp_(0, img0_shape[1])  # x2
    coords[:, 3].clamp_(0, img0_shape[0])  # y2
    coords[:, 4].clamp_(0, img0_shape[1])  # x3
    coords[:, 5].clamp_(0, img0_shape[0])  # y3
    coords[:, 6].clamp_(0, img0_shape[1])  # x4
    coords[:, 7].clamp_(0, img0_shape[0])  # y4
    coords[:, 8].clamp_(0, img0_shape[1])  # x5
    coords[:, 9].clamp_(0, img0_shape[0])  # y5
    
    return coords

In [6]:
def show_det_box(img, xywh, conf):
    # Draw detected bounding box

    h,w,c = img.shape
    
    tl = 1 or round(0.002 * (h + w) / 2) + 1  # line/font thickness
    
    # Convert YOLO coordinates to x1, y1, x2, y2 format
    x1 = int(xywh[0] * w - 0.5 * xywh[2] * w)
    y1 = int(xywh[1] * h - 0.5 * xywh[3] * h)
    x2 = int(xywh[0] * w + 0.5 * xywh[2] * w)
    y2 = int(xywh[1] * h + 0.5 * xywh[3] * h)
    
    # Draw detected bounding box
    cv2.rectangle(img, (x1,y1), (x2, y2), (0,255,0), thickness=tl, lineType=cv2.LINE_AA)

    # Put confidence threshold
    tf = max(tl - 1, 1)  # font thickness
    label = str(conf)[:5]
    cv2.putText(img, label, (x1, y1 - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)
    
    return img

In [12]:
def detect(model, image_path, save_path, device, imgsz=640, conf_thres=0.5, iou_thres=0.5, augment=None):
    # Detect face region and facial landmarks on given images

    # Load image and create a copy
    orgimg = cv2.imread(image_path)
    img0 = orgimg.copy()
    
    # Preprocess the image     
    stride = int(model.stride.max())    
    if imgsz <= 0:                    
        imgsz = dynamic_resize(img0.shape)
    imgsz = check_img_size(imgsz, s=64)
    img = letterbox(img0, imgsz)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).to(device)
    img = img.float()
    img /= 255.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    if augment is not None:
        pred = model(img, augment=True, aug_setting=augment)[0]
    else:
        pred = model(img, augment=False)[0]   
        
    # Apply non max suppression
    pred = non_max_suppression_face(pred, conf_thres, iou_thres)[0]
    gn = torch.tensor(img0.shape)[[1, 0, 1, 0]].to(device)                        # normalization gain whwh
    gn_lks = torch.tensor(img0.shape)[[1, 0, 1, 0, 1, 0, 1, 0, 1, 0]].to(device)  # normalization gain landmarks
    
    # Scale and store predictions
    boxes = []
    landmarks = []
    conf_scores = []
    h, w, c = img0.shape
    if pred is not None:
        pred[:, :4] = scale_coords(img.shape[2:], pred[:, :4], img0.shape).round()
        pred[:, 5:15] = scale_coords_landmarks(img.shape[2:], pred[:, 5:15], img0.shape).round()
        for j in range(pred.size()[0]):
            xywh = (xyxy2xywh(pred[j, :4].view(1, 4)) / gn).view(-1)
            xywh = xywh.data.cpu().numpy()
            conf = pred[j, 4].cpu().numpy()
            points = (pred[j, 5:15].view(1, 10) / gn_lks).view(-1).tolist()
            class_num = pred[j, 15].cpu().numpy()
            x1 = int(xywh[0] * w - 0.5 * xywh[2] * w)
            y1 = int(xywh[1] * h - 0.5 * xywh[3] * h)
            x2 = int(xywh[0] * w + 0.5 * xywh[2] * w)
            y2 = int(xywh[1] * h + 0.5 * xywh[3] * h)
                
            boxes.append([x1, y1, x2-x1, y2-y1])
            landmarks.append(points)
            conf_scores.append(conf)
            
            # Draw detected bounding box
            orgimg = show_det_box(orgimg, xywh, conf)
         
    # Save the image
    img = PIL.Image.fromarray(cv2.cvtColor(orgimg, cv2.COLOR_BGR2RGB))
    img.save(os.path.join(save_path, image_path.split("/")[-1]))
        
    return boxes, landmarks, conf_scores

In [8]:
exp_name = "test"     # experiment ID
iou_thres = 0.5           # IoU threshold
conf_thres = 0.5          # confidence threshold
img_size = 640            # image size

weights = "/workspace/yolov5-face-my/runs/train/exp1_l6/weights/best.pt"     # path to the model weights

# external dataset directories
anime_dir = '/workspace/selected-external/animefaces/'
art_dir = '/workspace/selected-external/artisticfaces/'
birds_dir = '/workspace/selected-external/birds/'
met_dir = '/workspace/selected-external/metfaces/'
pets_dir = '/workspace/selected-external/pets/'
turtles_dir = '/workspace/selected-external/turtles/'
tj_dir = '/workspace/selected-external/tj/'
fish_dir = '/workspace/selected-external/fish/'

# Get paths to the external dataset images
anime_paths = list(paths.list_images(os.path.join(anime_dir, 'images')))
art_paths = list(paths.list_images(os.path.join(art_dir, 'images')))
birds_paths = list(paths.list_images(os.path.join(birds_dir, 'images')))
met_paths = list(paths.list_images(os.path.join(met_dir, 'images')))
pets_paths = list(paths.list_images(os.path.join(pets_dir, 'images')))
turtles_paths = list(paths.list_images(os.path.join(turtles_dir, 'images')))
tj_paths = list(paths.list_images(os.path.join(tj_dir, 'images')))
fish_paths = list(paths.list_images(os.path.join(fish_dir, 'images')))

print("Anime Faces set size: {}". format(len(anime_paths)))
print("Artistic Faces set size: {}". format(len(art_paths)))
print("Birds Faces set size: {}". format(len(birds_paths)))
print("Met Faces set size: {}". format(len(met_paths)))
print("Pet Faces set size: {}". format(len(pets_paths)))
print("Turtle Faces set size: {}". format(len(turtles_paths)))
print("Tom & Jerry Faces set size: {}". format(len(tj_paths)))
print("Fish Faces set size: {}". format(len(tj_paths)))

Anime Faces set size: 160
Artistic Faces set size: 160
Birds Faces set size: 160
Met Faces set size: 160
Pet Faces set size: 160
Turtle Faces set size: 160
Tom & Jerry Faces set size: 160
Fish Faces set size: 160


In [16]:
# Create directories for saving images with detected bounding boxes
birds_res_path = "selected_external_results/{}/birds/".format(exp_name)
art_res_path = "selected_external_results/{}/artisticfaces/".format(exp_name)
turtles_res_path = "selected_external_results/{}/turtles/".format(exp_name)
anime_res_path = "selected_external_results/{}/animefaces/".format(exp_name)
pets_res_path = "selected_external_results/{}/pets/".format(exp_name)
met_res_path = "selected_external_results/{}/metfaces/".format(exp_name)
tj_res_path = "selected_external_results/{}/tj/".format(exp_name)
fish_res_path = "selected_external_results/{}/fish/".format(exp_name)

make_dir(birds_res_path)
make_dir(art_res_path)
make_dir(turtles_res_path)
make_dir(anime_res_path)
make_dir(pets_res_path)
make_dir(met_res_path)
make_dir(tj_res_path)
make_dir(fish_res_path)

[INFO] Directory  selected_external_results/test/birds/  created
[INFO] Directory  selected_external_results/test/artisticfaces/  created
[INFO] Directory  selected_external_results/test/turtles/  created
[INFO] Directory  selected_external_results/test/animefaces/  created
[INFO] Directory  selected_external_results/test/pets/  created
[INFO] Directory  selected_external_results/test/metfaces/  created
[INFO] Directory  selected_external_results/test/tj/  created
[INFO] Directory  selected_external_results/test/fish/  created


In [10]:
# Load weights 
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
model = attempt_load(weights, map_location=device)
device, weights

Fusing layers... 
None


(device(type='cuda', index=2),
 '/workspace/yolov5-face-my/runs/train/exp1_l6/weights/best.pt')

In [17]:
for i in range(160):
    detect(model, pets_paths[i], pets_res_path, device, img_size, conf_thres, iou_thres, augment=1)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [12]:
for i in range(160):
    detect(model, birds_paths[i], birds_res_path, device, img_size, conf_thres, iou_thres)

/workspace/external/birds/labels_eval/Artic_Tern_0027_141617.txt
[[249.0, 70.0, 68.0, 58.0]]
/workspace/external/birds/labels_eval/Black_Throated_Blue_Warbler_0004_161404.txt
[[303.0, 72.0, 106.0, 61.0]]
/workspace/external/birds/labels_eval/Bohemian_Waxwing_0028_796647.txt
[[191.0, 42.0, 79.0, 44.0]]
/workspace/external/birds/labels_eval/Hooded_Oriole_0091_90821.txt
[[299.0, 116.0, 137.0, 97.0]]
/workspace/external/birds/labels_eval/European_Goldfinch_0025_794647.txt
[[295.0, 131.0, 63.0, 84.0]]
/workspace/external/birds/labels_eval/Henslow_Sparrow_0108_796592.txt
[[158.0, 128.0, 79.0, 57.0]]
/workspace/external/birds/labels_eval/Prothonotary_Warbler_0040_173447.txt
[[199.0, 123.0, 75.0, 81.0]]
/workspace/external/birds/labels_eval/Yellow_Headed_Blackbird_0072_8606.txt
[[62.0, 107.0, 140.0, 50.0]]
/workspace/external/birds/labels_eval/Chipping_Sparrow_0076_108919.txt
[[157.0, 101.0, 62.0, 32.0]]
/workspace/external/birds/labels_eval/Red_Legged_Kittiwake_0019_795398.txt
[[52.0, 216.0, 

/workspace/external/birds/labels_eval/Hooded_Merganser_0068_79057.txt
[[106.0, 102.0, 135.0, 68.0]]
/workspace/external/birds/labels_eval/Red_Breasted_Merganser_0040_79207.txt
[[332.0, 96.0, 114.0, 74.0]]
/workspace/external/birds/labels_eval/Warbling_Vireo_0052_158534.txt
[[151.0, 118.0, 91.0, 44.0]]
/workspace/external/birds/labels_eval/Pine_Warbler_0065_172119.txt
[[0.0, -18.0, 371.0, 122.0]]
/workspace/external/birds/labels_eval/Pileated_Woodpecker_0098_180170.txt
[[75.0, 95.0, 101.0, 101.0]]
/workspace/external/birds/labels_eval/Spotted_Catbird_0020_796794.txt
[[280.0, 21.0, 119.0, 58.0]]
/workspace/external/birds/labels_eval/Red_Winged_Blackbird_0075_4953.txt
[[213.0, 185.0, 23.0, 11.0]]
/workspace/external/birds/labels_eval/Dark_Eyed_Junco_0008_68321.txt
[[270.0, 70.0, 50.0, 42.0]]
/workspace/external/birds/labels_eval/Rhinoceros_Auklet_0007_797521.txt
[[158.0, 56.0, 123.0, 53.0]]
/workspace/external/birds/labels_eval/Herring_Gull_0076_47497.txt
[[257.0, 15.0, 97.0, 72.0]]
/work

In [13]:
for i in range(160):
    detect(model, art_paths[i], art_res_path, device, img_size, conf_thres, iou_thres)

/workspace/external/artisticfaces/labels_eval/28.txt
[[53.0, 50.0, 154.0, 191.0]]
/workspace/external/artisticfaces/labels_eval/31.txt
[[61.0, 49.0, 133.0, 171.0]]
/workspace/external/artisticfaces/labels_eval/130.txt
[[49.0, 48.0, 153.0, 188.0]]
/workspace/external/artisticfaces/labels_eval/125.txt
[[59.0, 55.0, 145.0, 176.0]]
/workspace/external/artisticfaces/labels_eval/27.txt
[[53.0, 60.0, 165.0, 174.0]]
/workspace/external/artisticfaces/labels_eval/49.txt
[[43.0, 47.0, 172.0, 178.0]]
/workspace/external/artisticfaces/labels_eval/111.txt
[[54.0, 40.0, 153.0, 201.0]]
/workspace/external/artisticfaces/labels_eval/118.txt
[[56.0, 56.0, 144.0, 176.0]]
/workspace/external/artisticfaces/labels_eval/7.txt
[[60.0, 53.0, 138.0, 189.0]]
/workspace/external/artisticfaces/labels_eval/129.txt
[[54.0, 57.0, 153.0, 166.0]]
/workspace/external/artisticfaces/labels_eval/122.txt
[[55.0, 44.0, 152.0, 186.0]]
/workspace/external/artisticfaces/labels_eval/144.txt
[[74.0, 29.0, 115.0, 205.0]]
/workspace

/workspace/external/artisticfaces/labels_eval/75.txt
[[48.0, 36.0, 164.0, 197.0]]
/workspace/external/artisticfaces/labels_eval/43.txt
[[48.0, 26.0, 166.0, 210.0]]
/workspace/external/artisticfaces/labels_eval/85.txt
[[45.0, 37.0, 165.0, 198.0]]
/workspace/external/artisticfaces/labels_eval/41.txt
[[55.0, 52.0, 145.0, 172.0]]
/workspace/external/artisticfaces/labels_eval/69.txt
[[57.0, 51.0, 146.0, 169.0]]
/workspace/external/artisticfaces/labels_eval/86.txt
[[52.0, 10.0, 150.0, 206.0]]
/workspace/external/artisticfaces/labels_eval/104.txt
[[42.0, 30.0, 187.0, 218.0]]
/workspace/external/artisticfaces/labels_eval/45.txt
[[59.0, 40.0, 145.0, 182.0]]
/workspace/external/artisticfaces/labels_eval/26.txt
[[57.0, 47.0, 160.0, 188.0]]
/workspace/external/artisticfaces/labels_eval/40.txt
[[62.0, 46.0, 138.0, 179.0]]
/workspace/external/artisticfaces/labels_eval/30.txt
[[44.0, 57.0, 159.0, 172.0]]
/workspace/external/artisticfaces/labels_eval/152.txt
[[52.0, 70.0, 145.0, 156.0]]
/workspace/ext

In [14]:
for i in range(160):
    detect(model, met_paths[i], met_res_path, device, img_size, conf_thres, iou_thres)

/workspace/external/metfaces/labels_eval/image-746938.txt
[[235.0, 178.0, 136.0, 157.0]]
/workspace/external/metfaces/labels_eval/image-436627.txt
[[608.0, 609.0, 351.0, 423.0]]
/workspace/external/metfaces/labels_eval/image-548585.txt
[[576.0, 325.0, 168.0, 179.0]]
/workspace/external/metfaces/labels_eval/image-11602.txt
[[631.0, 540.0, 339.0, 363.0]]
/workspace/external/metfaces/labels_eval/image-438737.txt
[[663.0, 424.0, 228.0, 259.0]]
/workspace/external/metfaces/labels_eval/image-471908.txt
[[479.0, 440.0, 536.0, 615.0]]
/workspace/external/metfaces/labels_eval/image-12827.txt
[[747.0, 423.0, 337.0, 422.0]]
/workspace/external/metfaces/labels_eval/image-409000.txt
[[497.0, 484.0, 261.0, 279.0]]
/workspace/external/metfaces/labels_eval/image-10929.txt
[[487.0, 518.0, 220.0, 248.0]]
/workspace/external/metfaces/labels_eval/image-337431.txt
[[729.0, 562.0, 194.0, 205.0]]
/workspace/external/metfaces/labels_eval/image-438020.txt
[[486.0, 360.0, 242.0, 307.0]]
/workspace/external/metf

/workspace/external/metfaces/labels_eval/image-49892.txt
[[679.0, 435.0, 155.0, 151.0]]
/workspace/external/metfaces/labels_eval/image-11651.txt
[[512.0, 482.0, 282.0, 356.0]]
/workspace/external/metfaces/labels_eval/image-10176.txt
[[660.0, 313.0, 186.0, 210.0]]
/workspace/external/metfaces/labels_eval/image-15090.txt
[[620.0, 657.0, 183.0, 218.0]]
/workspace/external/metfaces/labels_eval/image-15307.txt
[[688.0, 748.0, 156.0, 198.0]]
/workspace/external/metfaces/labels_eval/image-436067.txt
[[443.0, 487.0, 307.0, 346.0]]
/workspace/external/metfaces/labels_eval/image-544690.txt
[[468.0, 1026.0, 637.0, 604.0]]
/workspace/external/metfaces/labels_eval/image-436314.txt
[[738.0, 725.0, 268.0, 361.0]]
/workspace/external/metfaces/labels_eval/image-436446.txt
[[467.0, 294.0, 240.0, 266.0]]
/workspace/external/metfaces/labels_eval/image-11720.txt
[[469.0, 608.0, 202.0, 215.0]]
/workspace/external/metfaces/labels_eval/image-437395.txt
[[643.0, 373.0, 199.0, 223.0]]
/workspace/external/metfac

In [15]:
for i in range(160):
    detect(model, turtles_paths[i], turtles_res_path, device, img_size, conf_thres, iou_thres)

/workspace/external/turtles/labels_eval/Image_450.txt
[[613.0, 216.0, 388.0, 420.0]]
/workspace/external/turtles/labels_eval/Image_1555.txt
[[465.0, 127.0, 281.0, 316.0]]
/workspace/external/turtles/labels_eval/Image_1056.txt
[[684.0, 285.0, 532.0, 503.0]]
/workspace/external/turtles/labels_eval/Image_1082.txt
[[284.0, 128.0, 147.0, 128.0]]
/workspace/external/turtles/labels_eval/Image_62.txt
[[533.0, 237.0, 165.0, 190.0]]
/workspace/external/turtles/labels_eval/Image_343.txt
[[515.0, 82.0, 144.0, 192.0]]
/workspace/external/turtles/labels_eval/Image_1514.txt
[[326.0, 277.0, 101.0, 69.0]]
/workspace/external/turtles/labels_eval/Image_1589.txt
[[409.0, 314.0, 248.0, 215.0]]
/workspace/external/turtles/labels_eval/Image_91.txt
[[1620.0, 539.0, 885.0, 706.0]]
/workspace/external/turtles/labels_eval/Image_306.txt
[[301.0, 239.0, 270.0, 224.0]]
/workspace/external/turtles/labels_eval/Image_935.txt
[[114.0, 374.0, 382.0, 327.0]]
/workspace/external/turtles/labels_eval/Image_1172.txt
[[511.0,

/workspace/external/turtles/labels_eval/Image_1462.txt
[[280.0, 165.0, 172.0, 168.0]]
/workspace/external/turtles/labels_eval/Image_1518.txt
[[486.0, 485.0, 119.0, 70.0]]
/workspace/external/turtles/labels_eval/Image_461.txt
[[327.0, 138.0, 271.0, 229.0]]
/workspace/external/turtles/labels_eval/Image_1358.txt
[[397.0, 179.0, 154.0, 130.0]]
/workspace/external/turtles/labels_eval/Image_1407.txt
[[600.0, 208.0, 312.0, 272.0]]
/workspace/external/turtles/labels_eval/Image_1626.txt
[[271.0, 331.0, 167.0, 110.0]]
/workspace/external/turtles/labels_eval/Image_629.txt
[[452.0, 234.0, 161.0, 163.0]]
/workspace/external/turtles/labels_eval/Image_102.txt
[[738.0, 162.0, 264.0, 194.0]]
/workspace/external/turtles/labels_eval/Image_1952.txt
[[427.0, 151.0, 255.0, 255.0]]
/workspace/external/turtles/labels_eval/Image_1486.txt
[[1624.0, 821.0, 307.0, 266.0]]
/workspace/external/turtles/labels_eval/Image_1534.txt
[[292.0, 167.0, 237.0, 199.0]]
/workspace/external/turtles/labels_eval/Image_362.txt
[[7

In [16]:
for i in range(160):
    detect(model, anime_paths[i], anime_res_path, device, img_size, conf_thres, iou_thres)

/workspace/external/animefaces/labels_eval/001706.txt
[[314.0, 165.0, 167.0, 167.0]]
/workspace/external/animefaces/labels_eval/002905.txt
[[96.0, 104.0, 173.0, 173.0], [303.0, 184.0, 197.0, 197.0]]
/workspace/external/animefaces/labels_eval/000914.txt
[[506.0, 312.0, 367.0, 367.0]]
/workspace/external/animefaces/labels_eval/006357.txt
[[289.0, 144.0, 306.0, 290.0]]
/workspace/external/animefaces/labels_eval/006021.txt
[[592.0, 603.0, 203.0, 203.0]]
/workspace/external/animefaces/labels_eval/006540.txt
[[492.0, 129.0, 158.0, 147.0], [727.0, 156.0, 149.0, 156.0], [273.0, 100.0, 160.0, 157.0]]
/workspace/external/animefaces/labels_eval/000026.txt
[[138.0, 210.0, 270.0, 270.0], [614.0, 418.0, 171.0, 171.0]]
/workspace/external/animefaces/labels_eval/003680.txt
[[464.0, 845.0, 105.0, 105.0]]
/workspace/external/animefaces/labels_eval/003816.txt
[[1566.0, 1253.0, 329.0, 329.0]]
/workspace/external/animefaces/labels_eval/004032.txt
[[319.0, 194.0, 110.0, 110.0]]
/workspace/external/animeface

/workspace/external/animefaces/labels_eval/005510.txt
[[336.0, 257.0, 259.0, 252.0]]
/workspace/external/animefaces/labels_eval/001673.txt
[[123.0, 172.0, 169.0, 168.0], [492.0, 68.0, 204.0, 200.0]]
/workspace/external/animefaces/labels_eval/001199.txt
[[336.0, 142.0, 239.0, 239.0]]
/workspace/external/animefaces/labels_eval/004246.txt
[[408.0, 35.0, 135.0, 136.0], [186.0, 60.0, 84.0, 82.0]]
/workspace/external/animefaces/labels_eval/004437.txt
[[708.0, 389.0, 214.0, 206.0]]
/workspace/external/animefaces/labels_eval/003376.txt
[[230.0, 316.0, 187.0, 187.0], [400.0, 61.0, 172.0, 172.0]]
/workspace/external/animefaces/labels_eval/001631.txt
[[241.0, 162.0, 168.0, 165.0]]
/workspace/external/animefaces/labels_eval/001271.txt
[[339.0, 125.0, 136.0, 136.0]]
/workspace/external/animefaces/labels_eval/000174.txt
[[197.0, 201.0, 318.0, 318.0]]
/workspace/external/animefaces/labels_eval/001259.txt
[[387.0, 186.0, 190.0, 190.0]]
/workspace/external/animefaces/labels_eval/003187.txt
[[989.0, 107

In [17]:
for i in range(160):
    detect(model, tj_paths[i], tj_res_path, device, img_size, conf_thres, iou_thres)

/workspace/external/tj/labels_eval/frame66.txt
[[440.0, 1.0, 191.0, 118.0]]
/workspace/external/tj/labels_eval/frame181.txt
[[431.0, 153.0, 424.0, 196.0]]
/workspace/external/tj/labels_eval/frame142.txt
[[531.0, 45.0, 260.0, 174.0]]
/workspace/external/tj/labels_eval/frame145.txt
[[791.0, 102.0, 225.0, 247.0]]
/workspace/external/tj/labels_eval/frame157.txt
[[346.0, 194.0, 140.0, 109.0]]
/workspace/external/tj/labels_eval/test158.txt
[[293.0, 1.0, 254.0, 216.0]]
/workspace/external/tj/labels_eval/test42.txt
[[712.0, 1.0, 96.0, 94.0]]
/workspace/external/tj/labels_eval/test181.txt
[[452.0, 478.0, 94.0, 65.0]]
/workspace/external/tj/labels_eval/frame206.txt
[[373.0, 57.0, 251.0, 168.0]]
/workspace/external/tj/labels_eval/test77.txt
[[309.0, 106.0, 219.0, 152.0]]
/workspace/external/tj/labels_eval/frame234.txt
[[323.0, 223.0, 332.0, 196.0], [892.0, 193.0, 92.0, 71.0]]
/workspace/external/tj/labels_eval/test152.txt
[[412.0, 109.0, 204.0, 195.0], [427.0, 25.0, 103.0, 77.0]]
/workspace/exter

/workspace/external/tj/labels_eval/test44.txt
[[435.0, 296.0, 121.0, 110.0]]
/workspace/external/tj/labels_eval/test65.txt
[[844.0, 30.0, 271.0, 251.0]]
/workspace/external/tj/labels_eval/frame53.txt
[[659.0, 200.0, 211.0, 161.0]]
/workspace/external/tj/labels_eval/frame154.txt
[[310.0, 46.0, 260.0, 310.0]]
/workspace/external/tj/labels_eval/frame124.txt
[[645.0, 152.0, 225.0, 133.0]]
/workspace/external/tj/labels_eval/frame240.txt
[[331.0, 211.0, 322.0, 221.0]]
/workspace/external/tj/labels_eval/test121.txt
[[954.0, 89.0, 186.0, 349.0]]
/workspace/external/tj/labels_eval/frame201.txt
[[438.0, 63.0, 340.0, 224.0]]
/workspace/external/tj/labels_eval/frame200.txt
[[421.0, 106.0, 165.0, 169.0]]
/workspace/external/tj/labels_eval/test19.txt
[[584.0, 219.0, 307.0, 250.0]]
/workspace/external/tj/labels_eval/test23.txt
[[591.0, 203.0, 291.0, 236.0]]
/workspace/external/tj/labels_eval/frame169.txt
[[375.0, 254.0, 392.0, 191.0]]
/workspace/external/tj/labels_eval/frame228.txt
[[353.0, 203.0, 35